# Analysing

Crea una copia del individuo en cada generación. 
Notebook enfocado a la visualización.

In [327]:
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt 
 

In [328]:
GENS = 200 #hay una generación más, la que contiene el PF
population = 40


In [329]:
df = pd.read_csv("../examples/fpp/test.csv")
df.head()

def clean_parents(item):
    item = list(eval(item))
    return pd.Series([item[0], item[1]])

df[["parent1","parent2"]]=df["parent"].apply(clean_parents)

In [330]:
df.drop(df[df.iter==GENS].index,inplace=True) #these samples are in the GENS+1 

In [331]:
df.loc[df.iter==GENS+1,"iter"]=GENS #we change the GENS+1 to GEN 

In [332]:
df[df.pf==False]

,iter,idx,n_gen,n_iter,rank,crowding,parent,mutate,mutate_rate,pf,parent1,parent2


In [333]:
# df.loc[df.pf=="nan","pf"]=False
df.pf.fillna(False,inplace=True)
df.loc[df.pf=="true","pf"]=True
df.loc[df.pf=="false","pf"]=False

In [334]:
print((df.pf==True).all())
df.head()

False


,iter,idx,n_gen,n_iter,rank,crowding,parent,mutate,mutate_rate,pf,parent1,parent2
0,1,1_12,1,1,0,inf,"['0', '0']",NaN,NaN,False,0,0
1,1,1_29,1,1,0,1.00000,"['0', '0']",NaN,NaN,False,0,0
2,1,1_39,1,1,0,inf,"['0', '0']",NaN,NaN,False,0,0
3,1,1_9,1,1,1,inf,"['0', '0']",NaN,NaN,False,0,0
4,1,1_38,1,1,1,0.55597,"['0', '0']",NaN,NaN,False,0,0


In [335]:
df.tail()

,iter,idx,n_gen,n_iter,rank,crowding,parent,mutate,mutate_rate,pf,parent1,parent2
8035,200,167_5,167,167,0,0.030031,"['152_11', '164_22']",NaN,NaN,True,152_11,164_22
8036,200,154_11,154,154,0,0.030031,"['150_4', '94_23']",True,unkown,True,150_4,94_23
8037,200,152_24,152,152,0,0.030031,"['144_20', '136_18']",True,unkown,True,144_20,136_18
8038,200,164_23,164,164,0,0.030031,"['152_6', '158_37']",True,unkown,True,152_6,158_37
8039,200,162_3,162,162,0,0.030031,"['158_39', '88_28']",True,unkown,True,158_39,88_28


In [336]:
np.unique(df.pf)

array([False,  True])

In [337]:
mask = df.groupby(by=["idx"],as_index=False)["iter"].agg([min,max])
mask["lifeIter"] = mask["max"]-mask["min"]
dfMutates = df[["idx","mutate"]].drop_duplicates()
dfMutates.fillna(False,inplace=True)
df2 = pd.merge(mask,dfMutates,on=["idx"],how="inner")

dft = pd.merge(df,df2,on=["idx"],how="inner")

In [338]:
dft["name"]=dft["idx"]+"_"+dft["iter"].astype(str)
print(dft.shape)

(8000, 17)


In [339]:
dft.head()

,iter,idx,n_gen,n_iter,rank,crowding,parent,mutate_x,mutate_rate,pf,parent1,parent2,min,max,lifeIter,mutate_y,name
0,1,1_12,1,1,0,inf,"['0', '0']",NaN,NaN,False,0,0,1,5,4,False,1_12_1
1,2,1_12,1,1,0,inf,"['0', '0']",NaN,NaN,False,0,0,1,5,4,False,1_12_2
2,3,1_12,1,1,0,0.123588,"['0', '0']",NaN,NaN,False,0,0,1,5,4,False,1_12_3
3,4,1_12,1,1,1,0.115418,"['0', '0']",NaN,NaN,False,0,0,1,5,4,False,1_12_4
4,5,1_12,1,1,4,0.106645,"['0', '0']",NaN,NaN,False,0,0,1,5,4,False,1_12_5


In [340]:
dft.tail()

,iter,idx,n_gen,n_iter,rank,crowding,parent,mutate_x,mutate_rate,pf,parent1,parent2,min,max,lifeIter,mutate_y,name
7995,200,199_27,199,199,0,0.030031,"['152_24', '195_39']",True,unkown,True,152_24,195_39,199,200,1,True,199_27_200
7996,199,199_39,199,199,0,0.030031,"['185_22', '198_35']",NaN,NaN,False,185_22,198_35,199,200,1,False,199_39_199
7997,200,199_39,199,199,0,0.058552,"['185_22', '198_35']",NaN,NaN,True,185_22,198_35,199,200,1,False,199_39_200
7998,200,200_15,200,200,0,0.046467,"['197_8', '187_27']",True,unkown,True,197_8,187_27,200,200,0,True,200_15_200
7999,200,200_32,200,200,0,0.030031,"['199_27', '197_15']",True,unkown,True,199_27,197_15,200,200,0,True,200_32_200


In [341]:
import networkx as nx
G = nx.Graph()
# G.add_nodes_from(dft[dft.iter==1]["idx"].values)
G.add_nodes_from(dft.name.values)
print(len(G.nodes))

8000


In [342]:
lastGeneration = dict(zip(dft[dft.iter==GENS].name,[True]*population))
iteration_att = dict(zip(dft.name,dft.iter))
rank_att = dict(zip(dft.name,dft["rank"]))
lifeiter_att = dict(zip(dft.name,dft["lifeIter"]))
pf_att = dict(zip(dft.name,dft["pf"].astype(int)))
mutate_att = dict(zip(dft.name,dft["mutate_y"].astype(int)))


nx.set_node_attributes(G,values=iteration_att,name="iter")
nx.set_node_attributes(G,values=rank_att,name="rank")
nx.set_node_attributes(G,values=lifeiter_att,name="lifeIter")
nx.set_node_attributes(G,values=pf_att,name="pf")
nx.set_node_attributes(G,values=lastGeneration,name="lastGen")
nx.set_node_attributes(G,values=mutate_att,name="mutate")

In [343]:
pf_att.values()

dict_values([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [344]:
print(len(G.nodes))

8000


In [345]:
posX, posY = dict(),dict()
ghost = dict()
lastpost = defaultdict(int)

for gen in range(1,GENS+1):
    dftmp = dft[dft["iter"]==gen].sort_values(["lifeIter"],ascending=False)
    for element in range(len(dftmp)):
        ind = dftmp.iloc[element]
        name = ind["name"]
        iter = ind["iter"]
        n_gen = int(ind["n_gen"])
        id = ind["idx"]
        
        # # Edge generation
        if n_gen==gen and gen>1: #tiene ancestors
            p1 = ind["parent1"]
            p2 = ind["parent2"]
            namep1 = p1+"_"+str(gen-1)
            namep2 = p2+"_"+str(gen-1)
            G.add_edge(namep1, name)
            G.add_edge(namep2, name)

        if gen > n_gen: #individuo ya creado
            # Crear nuevo individuo GHOST en la nueva posición
            ghost[name]= True
            posX[name] = posX["%s_%i"%(id,gen-1)]
            posY[name] = -float(gen)
        else:
            # individuo nuevo
            if name=="4_2_4":
                print("\tHERE2")
            
            ghost[name]=False

            # Control Position 
            if gen == 1 :
                posX[name] = lastpost[gen]
                # lastpost[gen] = element+1
            elif gen == 2:
                posX[name] = -(lastpost[gen]+1)
            else: #3 o more generations
                if lastpost[gen]==0:
                    lastpost[gen]=lastpost[gen-1]+1
                posX[name] = - lastpost[gen]
                # lastpost[gen] = element+1
            lastpost[gen] = lastpost[gen]+1
            posY[name] = -float(gen)

    
print(len(G.nodes))

8000


In [346]:
nx.set_node_attributes(G,values=posX,name="posx")
nx.set_node_attributes(G,values=posY,name="posy")
nx.set_node_attributes(G,values=ghost,name="ghost")
print(len(G.nodes))


8000


In [347]:
nx.write_gexf(G, "test_g%i_p%i.gexf"%(GENS,population))
# on Gephy::Layout
# % which dot
# /opt/homebrew/bin/dot
